# Initial Imports - DO NOT DELETE

## Special imports for cookiecutter

In [1]:
# DO NOT REMOVE THESE
%load_ext autoreload
%autoreload 1
#%aimport src.base

The autoreload module is not an IPython extension.


UsageError: Line magic function `%autoreload` not found.


In [2]:
# DO NOT REMOVE This
%reload_ext autoreload

In [3]:
## DO NOT REMOVE
## import local src module -
## src in this project will contain all your local code
## clean_data.py, model.py, visualize.py, custom.py

#import src.base with alias to avoid namespace clashes

#from src import base as fis

In [4]:
#fis.test_base()

## Import libraries

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose

remote = True

In [12]:
localpath = '../data/processed/'
filename = 'zillow_chicago_df.csv'


# get the original data and save it immediately
# if remote:
#     print('Retrieving remote data')
#     remotefileurl = 'https://raw.githubusercontent.com/learn-co-students/dc_ds_04_22_19/master/module_4/project/data/zillow_data.csv'
#     raw_df = pd.read_csv(remotefileurl)
    
#     # Save the raw data locally so we don't have to use the repo again.
#     raw_df.to_csv(localpath+filename, index=False)
    
#     #set remote to false so we don't always read from the repo
#     remote=False
# else:
#     raw_df = pd.read_csv(localpath+filename)

raw_df = pd.read_csv(localpath+filename)
zillow_df = raw_df.copy()

# Data Collection

In [13]:
# This lets you see all of the columns in the df
# pd.options.display.max_columns = None

In [14]:
zillow_df.tail()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
14718,58333,1338,Ashfield,MA,Greenfield Town,Franklin,14719,94600.0,94300.0,94000.0,...,216800,217700,218600,218500,218100,216400,213100,209800,209200,209300
14719,59107,3293,Woodstock,NH,Claremont,Grafton,14720,92700.0,92500.0,92400.0,...,202100,208400,212200,215200,214300,213100,213700,218300,222700,225800
14720,75672,40404,Berea,KY,Richmond,Madison,14721,57100.0,57300.0,57500.0,...,121800,122800,124600,126700,128800,130600,131700,132500,133000,133400
14721,93733,81225,Mount Crested Butte,CO,NaN,Gunnison,14722,191100.0,192400.0,193700.0,...,662800,671200,682400,695600,695500,694700,706400,705300,681500,664400
14722,95851,89155,Mesquite,NV,Las Vegas,Clark,14723,176400.0,176300.0,176100.0,...,333800,336400,339700,343800,346800,348900,350400,353000,356000,357200


In [5]:
zillow_df.shape

(14723, 272)

In [17]:
zillow_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 43 columns):
Unnamed: 0    265 non-null int64
date          265 non-null object
60657         265 non-null float64
60614         265 non-null float64
60640         265 non-null float64
60647         265 non-null float64
60618         265 non-null float64
60613         265 non-null float64
60629         265 non-null float64
60625         265 non-null float64
60619         265 non-null float64
60610         265 non-null float64
60611         58 non-null float64
60620         265 non-null float64
60634         265 non-null float64
60626         265 non-null float64
60649         265 non-null float64
60616         265 non-null float64
60622         265 non-null float64
60615         265 non-null float64
60641         265 non-null float64
60660         265 non-null float64
60608         265 non-null float64
60638         265 non-null float64
60639         265 non-null float64
60630         265 non-nu

In [23]:
zillow_df.describe()

,60657,60614,60640,60647,60618,60613,60629,60625,60619,60610,...,60612,60607,60659,60652,60631,60656,60655,60646,60642,60633
count,2.650000e+02,2.650000e+02,265.000000,265.000000,265.000000,2.650000e+02,265.000000,265.000000,265.000000,2.650000e+02,...,265.000000,265.000000,265.000000,265.000000,265.000000,265.000000,265.000000,265.000000,265.000000,265.000000
mean,7.439789e+05,9.741392e+05,553078.867925,293135.471698,316441.132075,7.162766e+05,145572.452830,330923.018868,120266.037736,7.800868e+05,...,243598.113208,417041.132075,296586.792453,150041.509434,298997.358491,263675.094340,192772.830189,333294.716981,424144.528302,105211.698113
std,1.908211e+05,2.312259e+05,167499.870439,97412.855511,96112.225708,1.972157e+05,43406.557113,93507.227269,28518.404102,1.738772e+05,...,76955.057946,99293.470399,73280.550661,34232.310373,66842.258004,60514.191161,43548.942529,74299.319724,98538.344993,23603.241442
min,3.342000e+05,4.947000e+05,216500.000000,118700.000000,130400.000000,2.979000e+05,83600.000000,145400.000000,69800.000000,4.257000e+05,...,100900.000000,210700.000000,158100.000000,92100.000000,168200.000000,156100.000000,110800.000000,183200.000000,217700.000000,75400.000000
25%,6.734000e+05,8.782000e+05,455200.000000,230000.000000,262900.000000,6.295000e+05,113300.000000,274700.000000,103500.000000,7.262000e+05,...,196500.000000,370600.000000,259900.000000,127300.000000,258900.000000,220300.000000,153000.000000,297100.000000,386000.000000,85600.000000
50%,7.756000e+05,1.006300e+06,585900.000000,302500.000000,328900.000000,7.504000e+05,135600.000000,344300.000000,118600.000000,8.084000e+05,...,236800.000000,433000.000000,305800.000000,144200.000000,304600.000000,265200.000000,199700.000000,340600.000000,438300.000000,98300.000000
75%,8.899000e+05,1.162800e+06,686100.000000,367700.000000,391900.000000,8.478000e+05,164900.000000,399800.000000,133400.000000,9.058000e+05,...,312800.000000,491000.000000,354100.000000,172400.000000,342100.000000,300600.000000,223900.000000,382100.000000,491500.000000,121400.000000
max,1.033800e+06,1.308500e+06,798000.000000,475100.000000,464700.000000,1.060200e+06,239100.000000,464200.000000,180100.000000,1.050800e+06,...,361400.000000,579700.000000,418900.000000,218700.000000,416100.000000,383300.000000,262900.000000,467600.000000,606700.000000,153600.000000


In [22]:
zillow_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [24]:
zillow_df.set_index('date', inplace=True)

In [25]:
zillow_df.head()

,60657,60614,60640,60647,60618,60613,60629,60625,60619,60610,...,60612,60607,60659,60652,60631,60656,60655,60646,60642,60633
date,,,,,,,,,,,,,,,,,,,,,
1996-04-01,334200.0,498100.0,216500.0,122700.0,142600.0,297900.0,93400.0,148900.0,76400.0,435200.0,...,100900.0,215600.0,165400.0,103700.0,180900.0,166300.0,125700.0,197300.0,224100.0,77900.0
1996-05-01,335400.0,500900.0,216700.0,122800.0,143100.0,300400.0,94100.0,149300.0,77000.0,435200.0,...,101400.0,215100.0,165300.0,104400.0,182200.0,166900.0,126300.0,198900.0,223300.0,78200.0
1996-06-01,336500.0,503100.0,216900.0,122800.0,143400.0,302600.0,94600.0,149600.0,77400.0,435100.0,...,101900.0,214500.0,165000.0,104700.0,183100.0,167200.0,126600.0,200100.0,222400.0,78300.0
1996-07-01,337600.0,504600.0,217000.0,122700.0,143300.0,304700.0,94800.0,149700.0,77700.0,434600.0,...,102300.0,213900.0,164600.0,104700.0,183500.0,167100.0,126500.0,200800.0,221500.0,78500.0
1996-08-01,338500.0,505500.0,217100.0,122400.0,142900.0,306600.0,94700.0,149700.0,77700.0,433900.0,...,102700.0,213200.0,164100.0,104400.0,183400.0,166900.0,126100.0,201000.0,220500.0,78500.0


# Title

## Subtitle